# Imports and Data overview

In [1]:
train_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/raw/train.csv'
test_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/raw/test.csv'
sub_url = 'https://raw.githubusercontent.com/sidt-ai/data-science-competitions/main/machinehack/ode_to_code_2022/data/raw/sample_submission.csv'

In [2]:
!pip install --upgrade --quiet optuna

In [3]:
!pip install --quiet xgboost==1.5

In [4]:
import time 

import numpy as np
from scipy.stats import mode
import pandas as pd
pd.set_option("precision", 4)

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif 
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold

import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBClassifier

SEED = 2311

In [5]:
train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

In [6]:
train.head()

,edible-poisonous,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,p,5.75,x,n,t,s,w,6.13,14.93,w,f,f,d,a
1,p,2.27,o,g,f,f,f,0.00,0.00,f,f,f,d,s
2,p,5.59,c,y,t,x,p,5.47,11.27,w,f,f,d,a
3,p,0.74,x,p,f,a,n,4.33,0.98,k,f,f,d,a
4,p,9.60,x,n,f,a,w,10.84,28.52,w,f,f,d,u


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42748 entries, 0 to 42747
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   edible-poisonous      42748 non-null  object 
 1   cap-diameter          42748 non-null  float64
 2   cap-shape             42748 non-null  object 
 3   cap-color             42748 non-null  object 
 4   does-bruise-or-bleed  42748 non-null  object 
 5   gill-attachment       35808 non-null  object 
 6   gill-color            42748 non-null  object 
 7   stem-height           42748 non-null  float64
 8   stem-width            42748 non-null  float64
 9   stem-color            42748 non-null  object 
 10  has-ring              42748 non-null  object 
 11  ring-type             41029 non-null  object 
 12  habitat               42748 non-null  object 
 13  season                42748 non-null  object 
dtypes: float64(3), object(11)
memory usage: 4.6+ MB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18321 entries, 0 to 18320
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   edible-poisonous      18321 non-null  object 
 1   cap-diameter          18321 non-null  float64
 2   cap-shape             18321 non-null  object 
 3   cap-color             18321 non-null  object 
 4   does-bruise-or-bleed  18321 non-null  object 
 5   gill-attachment       15377 non-null  object 
 6   gill-color            18321 non-null  object 
 7   stem-height           18321 non-null  float64
 8   stem-width            18321 non-null  float64
 9   stem-color            18321 non-null  object 
 10  has-ring              18321 non-null  object 
 11  ring-type             17569 non-null  object 
 12  habitat               18321 non-null  object 
 13  season                0 non-null      float64
dtypes: float64(4), object(10)
memory usage: 2.0+ MB


In [9]:
test.drop(['season'], axis=1, inplace=True)

# Missing values

In [10]:
train.isna().sum()

edible-poisonous           0
cap-diameter               0
cap-shape                  0
cap-color                  0
does-bruise-or-bleed       0
gill-attachment         6940
gill-color                 0
stem-height                0
stem-width                 0
stem-color                 0
has-ring                   0
ring-type               1719
habitat                    0
season                     0
dtype: int64

In [11]:
test.isna().sum()

edible-poisonous           0
cap-diameter               0
cap-shape                  0
cap-color                  0
does-bruise-or-bleed       0
gill-attachment         2944
gill-color                 0
stem-height                0
stem-width                 0
stem-color                 0
has-ring                   0
ring-type                752
habitat                    0
dtype: int64

In [12]:
train['gill-attachment'].unique(), test['gill-attachment'].unique()

(array(['s', 'f', 'x', 'a', 'e', nan, 'd', 'p'], dtype=object),
 array(['x', 'a', nan, 'd', 'e', 'p', 's', 'f'], dtype=object))

In [13]:
train['ring-type'].unique(), test['ring-type'].unique()

(array(['f', 'p', nan, 'l', 'z', 'g', 'm', 'e', 'r'], dtype=object),
 array([nan, 'l', 'f', 'm', 'e', 'z', 'r', 'p', 'g'], dtype=object))

According to the data description, 'f' corresponds to 'none' for both these features with missing values.  

Thus, we will fill missing values with 'f'.

In [14]:
train.fillna('f', axis=1, inplace=True)
test.fillna('f', axis=1, inplace=True)

# EDA and Feature Engineering

In [15]:
train['season'].value_counts()

a    21103
u    16055
w     3698
s     1892
Name: season, dtype: int64

In [16]:
features = [f for f in train.columns if f not in ('season', 'fold')]
num_features = ['cap-diameter', 'stem-height', 'stem-width']
cat_features = [f for f in features if f not in num_features]
cat_mask = [f in cat_features for f in features]

In [17]:
train[cat_features].describe()

,edible-poisonous,cap-shape,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-color,has-ring,ring-type,habitat
count,42748,42748,42748,42748,42748,42748,42748,42748,42748,42748
unique,2,7,12,2,7,12,13,2,8,8
top,p,x,n,f,f,w,w,f,f,d
freq,23707,18884,16940,35335,9373,12930,16014,32138,35583,30934


In [18]:
train[num_features].describe()

,cap-diameter,stem-height,stem-width
count,42748.0000,42748.0000,42748.0000
mean,6.7120,6.5804,12.1340
std,5.2038,3.3684,10.0589
min,0.3800,0.0000,0.0000
25%,3.4800,4.6400,5.1900
50%,5.8400,5.9500,10.1400
75%,8.5400,7.7400,16.5300
max,62.3400,33.9200,103.9100


In [19]:
train[cat_features] = train[cat_features].astype('category')
test[cat_features] = test[cat_features].astype('category')

In [20]:
target_labels = LabelEncoder()
train['season'] = target_labels.fit_transform(train['season'])

# Hyperparameter tuning

In [21]:
base_params = {
    'objective': 'multi:softmax',
    'booster': 'gbtree',
    'use_label_encoder': False,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'enable_categorical': True,
    'n_estimators': 5000,
    'eval_metric': 'mlogloss',
    'random_state': SEED,
    'verbosity': 0
}

In [22]:
def objective(trial, xtrain, ytrain, xval, yval, base_params):
  param_grid = {
    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
    'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
    'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
    'max_depth': trial.suggest_int('max_depth', 3, 12)
  }

  model = XGBClassifier(**base_params, **param_grid)
    
  model.fit(
    xtrain, ytrain,
    eval_set=[(xval, yval)],
    early_stopping_rounds=100,
    callbacks=[
      XGBoostPruningCallback(trial, 'validation_0-mlogloss')
    ],
    verbose=100
  )
    
  val_predictions = model.predict(xval)
    
  return accuracy_score(yval, val_predictions)

In [23]:
xtrain, xval, ytrain, yval = train_test_split(train[features], train['season'],
                                              test_size=0.2,
                                              stratify=train['season'],
                                              shuffle=True,
                                              random_state=SEED)

In [24]:
study = optuna.create_study(direction='minimize', 
                            sampler=TPESampler(), 
                            study_name='MH_ode_to_code')

[I 2022-01-16 10:44:05,260] A new study created in memory with name: MH_ode_to_code


In [25]:
%%time
study.optimize(
    lambda trial: objective(trial, xtrain, ytrain, xval, yval, base_params),
    n_trials=50
)

[0]	validation_0-mlogloss:1.33675
[100]	validation_0-mlogloss:0.86464
[200]	validation_0-mlogloss:0.81330
[300]	validation_0-mlogloss:0.79058
[400]	validation_0-mlogloss:0.77745
[500]	validation_0-mlogloss:0.77128
[600]	validation_0-mlogloss:0.76786
[700]	validation_0-mlogloss:0.76606
[800]	validation_0-mlogloss:0.76556
[900]	validation_0-mlogloss:0.76487
[1000]	validation_0-mlogloss:0.76464
[1100]	validation_0-mlogloss:0.76543
[1106]	validation_0-mlogloss:0.76547


[I 2022-01-16 10:44:10,306] Trial 0 finished with value: 0.5214035087719299 and parameters: {'learning_rate': 0.11437703735509057, 'subsample': 0.6677718916854287, 'reg_lambda': 2.72715881041597e-05, 'reg_alpha': 2.891775740328728e-06, 'max_depth': 3}. Best is trial 0 with value: 0.5214035087719299.


[0]	validation_0-mlogloss:1.27501
[100]	validation_0-mlogloss:0.77262
[183]	validation_0-mlogloss:0.79619


[I 2022-01-16 10:44:13,435] Trial 1 finished with value: 0.5132163742690058 and parameters: {'learning_rate': 0.20901162959275282, 'subsample': 0.649101046910588, 'reg_lambda': 1.3454201242261047e-07, 'reg_alpha': 0.00033639349391350226, 'max_depth': 7}. Best is trial 1 with value: 0.5132163742690058.


[0]	validation_0-mlogloss:1.37665
[100]	validation_0-mlogloss:0.99473
[200]	validation_0-mlogloss:0.91006
[300]	validation_0-mlogloss:0.87165
[400]	validation_0-mlogloss:0.84685
[500]	validation_0-mlogloss:0.82944
[600]	validation_0-mlogloss:0.81581
[700]	validation_0-mlogloss:0.80493
[800]	validation_0-mlogloss:0.79640
[900]	validation_0-mlogloss:0.78974
[1000]	validation_0-mlogloss:0.78412
[1100]	validation_0-mlogloss:0.77957
[1200]	validation_0-mlogloss:0.77570
[1300]	validation_0-mlogloss:0.77280
[1400]	validation_0-mlogloss:0.77047
[1500]	validation_0-mlogloss:0.76853
[1600]	validation_0-mlogloss:0.76709
[1700]	validation_0-mlogloss:0.76561
[1800]	validation_0-mlogloss:0.76441
[1900]	validation_0-mlogloss:0.76350
[2000]	validation_0-mlogloss:0.76289
[2100]	validation_0-mlogloss:0.76245
[2200]	validation_0-mlogloss:0.76212
[2300]	validation_0-mlogloss:0.76150
[2400]	validation_0-mlogloss:0.76142
[2500]	validation_0-mlogloss:0.76116
[2585]	validation_0-mlogloss:0.76120


...

[0]	validation_0-mlogloss:1.23048


[I 2022-01-16 10:45:12,071] Trial 48 pruned. Trial was pruned at iteration 46.


[0]	validation_0-mlogloss:1.27269


[I 2022-01-16 10:45:12,148] Trial 49 pruned. Trial was pruned at iteration 0.


CPU times: user 1min 7s, sys: 1.35 s, total: 1min 8s
Wall time: 1min 6s


In [26]:
print(f'Best value (Accuracy): {study.best_value:.5f}')

best_params = study.best_params
print('Best params:')
for key, value in best_params.items():
    print(f'\t{key}: {value}')

Best value (Accuracy): 0.51322
Best params:
	learning_rate: 0.20901162959275282
	subsample: 0.649101046910588
	reg_lambda: 1.3454201242261047e-07
	reg_alpha: 0.00033639349391350226
	max_depth: 7


# Cross-validation + Inference

In [27]:
N_SPLITS = 5
train['fold'] = -1

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

for fold, (_, val_idx) in enumerate(skf.split(X=train, y=train['season'])):
  train.loc[val_idx, 'fold'] = fold

In [38]:
def custom_cross_val_predict(train, test, features, model):
  oof_preds = {}
  test_preds = []
  scores = []
    
  cv_start = time.time()
    
  for fold in range(N_SPLITS):
    print('-' * 40)
        
    xtrain = train[train.fold != fold].reset_index(drop=True)
    ytrain = xtrain.season

    xval = train[train.fold == fold].reset_index(drop=True)
    yval = xval.season
    val_idx = xval.index.tolist()
    
    fold_start = time.time()
        
    model.fit(
      xtrain[features], ytrain,
      eval_set=[(xval[features], yval)],
      early_stopping_rounds=100,
      verbose=100
    )
    
    val_preds = model.predict(xval[features])
    oof_preds.update(dict(zip(val_idx, val_preds)))
    acc = accuracy_score(yval, val_preds)
    scores.append(acc)
        
    fold_end = time.time()
        
    print(f'Fold #{fold}: Accuracy = {acc:.5f} \
    [Time: {fold_end - fold_start:.2f}s]')
        
    test_preds.append(model.predict(test[features]))
        
    cv_end = time.time()
  
  print(f'Average accuracy = {np.mean(scores):.5f} \
  with std. dev. = {np.std(scores):.5f}')
  
  print(f'[Total time: {cv_end - cv_start:.2f}s]')
    
  oof_preds = pd.DataFrame.from_dict(oof_preds, orient='index').reset_index()
  test_preds = mode(np.column_stack(test_preds), axis=1).mode
    
  return oof_preds, test_preds

In [39]:
model = XGBClassifier(**base_params, **best_params)

In [40]:
oof_preds, test_preds = custom_cross_val_predict(train, test, features, model)

----------------------------------------
[0]	validation_0-mlogloss:1.27623
[100]	validation_0-mlogloss:0.77336
[173]	validation_0-mlogloss:0.79483
Fold #0: Accuracy = 0.51719     [Time: 3.07s]
----------------------------------------
[0]	validation_0-mlogloss:1.27657
[100]	validation_0-mlogloss:0.77231
[174]	validation_0-mlogloss:0.79569
Fold #1: Accuracy = 0.51240     [Time: 2.94s]
----------------------------------------
[0]	validation_0-mlogloss:1.27611
[100]	validation_0-mlogloss:0.77073
[181]	validation_0-mlogloss:0.79287
Fold #2: Accuracy = 0.51357     [Time: 3.05s]
----------------------------------------
[0]	validation_0-mlogloss:1.27627
[100]	validation_0-mlogloss:0.77229
[169]	validation_0-mlogloss:0.79290
Fold #3: Accuracy = 0.51796     [Time: 2.85s]
----------------------------------------
[0]	validation_0-mlogloss:1.27483
[100]	validation_0-mlogloss:0.77258
[174]	validation_0-mlogloss:0.79662
Fold #4: Accuracy = 0.51374     [Time: 2.93s]
Average accuracy = 0.51497   with s

In [43]:
sub1 = pd.read_csv(sub_url)
sub1['season'] = target_labels.inverse_transform(test_preds.ravel())
sub1.to_csv('sub1.csv', index=False)

!head sub1.csv

season
s
u
u
u
u
u
a
u
a
